In [1]:
%matplotlib inline
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import netCDF4 as nc
import numpy as np
from Load_data import Data_from_nc
import xarray as xr
import math
import gc

def get_data_from_mat(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
#     var[var > 1000] = np.nan
#     var[var < -1000] = np.nan
    var = var[:, 29:] 
    return var

def get_data_from_mat_for_calcu(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, 29:]
    return var.T

data_land_mask = Data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/surfdata_0.9x1.25_hist_78pfts_CMIP6_simyr1850_c201126.nc')
data_mask = data_land_mask.load_variable('PFTDATA_MASK')
data_mask = data_mask[29:, :]

data_surface = Data_from_nc(\
                            '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/' + \
                            'surfdata_irrigation_method.nc'\
                           )   #load the data

ar6_region = get_data_from_mat_for_calcu('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/ar6_region.mat','ar6_region')
irr_diff = get_data_from_mat_for_calcu('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/irr_diff_out.mat', 'irr_diff_out')

data_irrigation_method = data_surface.load_variable('irrigation_method')

data_lon = data_surface.load_variable('LONGXY')
data_lon = data_lon[0,:]
data_lat = data_surface.load_variable('LATIXY')
data_lat = data_lat[29:,0]
data_lat = data_lat[:]
data_time = np.array(range(262800))

def get_data_for_this_study_1(metric, period, exp, ens, irr_low, irr_high, irr_diff):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_1.mat', metric)
    
    data_out = np.array(data_out)
    data_out[:, irr_diff< irr_low] = np.nan
    data_out[:, irr_diff>=irr_high] = np.nan
    return data_out

def get_data_for_this_study_2(metric, period, exp, ens, irr_low, irr_high, irr_diff):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_2.mat', metric)
    data_out = np.array(data_out)
    data_out[:, irr_diff< irr_low] = np.nan
    data_out[:, irr_diff>=irr_high] = np.nan
    return data_out

AREA = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/input_data/AREA.mat', 'AREA') # Area for CLM_data
AREA = AREA.T

def get_hour_km2_5_years(data_period, qctl):
    data_time_one_year = np.array(range(2920))
    hour_km_2_array = np.zeros([5])
    
    
    data_period_year0 = data_period[0 : 2920] 
    data_xr_new = xr.DataArray(data_period_year0, coords={'time': data_time_one_year, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    qscen=xr.where(data_xr_new>qctl,3,0) # three hours resolution
    sumscen=qscen.sum('time')
    hour_km_2_ = sumscen * AREA
    hour_km_2 = np.nansum(hour_km_2_)
    hour_km_2_array[0] = hour_km_2
    


    data_period_year1 = data_period[2920 : 2920+2920]
    data_xr_new = xr.DataArray(data_period_year1, coords={'time': data_time_one_year, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    qscen=xr.where(data_xr_new>qctl,3,0) # three hours resolution
    sumscen=qscen.sum('time')
    hour_km_2_ = sumscen * AREA
    hour_km_2 = np.nansum(hour_km_2_)
    hour_km_2_array[1] = hour_km_2
    


    data_period_year2 = data_period[2920+2920 : 2920+2920+2920]
    data_xr_new = xr.DataArray(data_period_year2, coords={'time': data_time_one_year, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    qscen=xr.where(data_xr_new>qctl,3,0) # three hours resolution
    sumscen=qscen.sum('time')
    hour_km_2_ = sumscen * AREA
    hour_km_2 = np.nansum(hour_km_2_)
    hour_km_2_array[2] = hour_km_2
    


    data_period_year3 = data_period[2920+2920+2920 : 2920+2920+2920+2920]
    data_xr_new = xr.DataArray(data_period_year3, coords={'time': data_time_one_year, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    qscen=xr.where(data_xr_new>qctl,3,0) # three hours resolution
    sumscen=qscen.sum('time')
    hour_km_2_ = sumscen * AREA
    hour_km_2 = np.nansum(hour_km_2_)
    hour_km_2_array[3] = hour_km_2
    


    data_period_year4 = data_period[2920+2920+2920+2920 : 2920+2920+2920+2920+2920]
    if data_period_year4.shape[0] <2920:
        hour_km_2_array[4]  = 0
    else:
    
        data_xr_new = xr.DataArray(data_period_year4, coords={'time': data_time_one_year, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
        qscen=xr.where(data_xr_new>qctl,3,0) # three hours resolution
        sumscen=qscen.sum('time')
        hour_km_2_ = sumscen * AREA
        hour_km_2 = np.nansum(hour_km_2_)
        hour_km_2_array[4] = hour_km_2
    
    return hour_km_2_array
    
def plot_line(ax, index, data_mean, data_min, data_max, title, title1, label, color, ylabel, xlabel, marker):
    ax.text(-0.05, 1.05, index, color='dimgrey', fontsize=12, transform=ax.transAxes, weight='bold')
    ax.plot(range(1901, 2015), data_mean, label = label, color = color, linewidth = 1, marker = marker, markersize=1.2, alpha=0.8)
    ax.fill_between(range(1901, 2015), data_min, data_max, color = color, alpha=0.2)
    plt.title(title, loc='right')
    plt.title(title1, loc='left')
    plt.ylabel(ylabel, fontsize = 14)
    plt.xlabel(xlabel, fontsize = 14)
    plt.yticks(fontsize = 12)
    plt.xticks(fontsize=12)
    plt.legend()
    
def get_hour_km2_4_years(data_period, qctl):
    data_time_one_year = np.array(range(2920))
    hour_km_2_array = np.zeros([4])
    
    
    data_period_year0 = data_period[0 : 2920] 
    data_xr_new = xr.DataArray(data_period_year0, coords={'time': data_time_one_year, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    qscen=xr.where(data_xr_new>qctl,3,0) # three hours resolution
    sumscen=qscen.sum('time')
    hour_km_2_ = sumscen * AREA
    hour_km_2 = np.nansum(hour_km_2_)
    hour_km_2_array[0] = hour_km_2
    


    data_period_year1 = data_period[2920 : 2920+2920]
    data_xr_new = xr.DataArray(data_period_year1, coords={'time': data_time_one_year, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    qscen=xr.where(data_xr_new>qctl,3,0) # three hours resolution
    sumscen=qscen.sum('time')
    hour_km_2_ = sumscen * AREA
    hour_km_2 = np.nansum(hour_km_2_)
    hour_km_2_array[1] = hour_km_2
    


    data_period_year2 = data_period[2920+2920 : 2920+2920+2920]
    data_xr_new = xr.DataArray(data_period_year2, coords={'time': data_time_one_year, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    qscen=xr.where(data_xr_new>qctl,3,0) # three hours resolution
    sumscen=qscen.sum('time')
    hour_km_2_ = sumscen * AREA
    hour_km_2 = np.nansum(hour_km_2_)
    hour_km_2_array[2] = hour_km_2
    
    data_period_year3 = data_period[2920+2920+2920 : 2920+2920+2920+2920]
    if data_period_year3.shape[0] <2920:
        hour_km_2_array[3]  = 0
    else:
        
        data_xr_new = xr.DataArray(data_period_year3, coords={'time': data_time_one_year, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
        qscen=xr.where(data_xr_new>qctl,3,0) # three hours resolution
        sumscen=qscen.sum('time')
        hour_km_2_ = sumscen * AREA
        hour_km_2 = np.nansum(hour_km_2_)
        hour_km_2_array[3] = hour_km_2
    
    return hour_km_2_array

def all_delta_irr(metric, perc, unit, variable_to_show, irr_low, irr_high):
    data_NOI01_1901_1910_1 = get_data_for_this_study_1(metric, '1901_1910', 'NOI', '01', irr_low, irr_high, irr_diff)
    data_NOI01_1901_1910_2 = get_data_for_this_study_2(metric, '1901_1910', 'NOI', '01', irr_low, irr_high, irr_diff)

    
    data_NOI01_1911_1920_1 = get_data_for_this_study_1(metric, '1911_1920', 'NOI', '01', irr_low, irr_high, irr_diff)
    data_NOI01_1911_1920_2 = get_data_for_this_study_2(metric, '1911_1920', 'NOI', '01', irr_low, irr_high, irr_diff)

    
    
    data_NOI01_1921_1930_1 = get_data_for_this_study_1(metric, '1921_1930', 'NOI', '01', irr_low, irr_high, irr_diff)
    data_NOI01_1921_1930_2 = get_data_for_this_study_2(metric, '1921_1930', 'NOI', '01', irr_low, irr_high, irr_diff)

    
    data_NOI02_1901_1910_1 = get_data_for_this_study_1(metric, '1901_1910', 'NOI', '02', irr_low, irr_high, irr_diff)
    data_NOI02_1901_1910_2 = get_data_for_this_study_2(metric, '1901_1910', 'NOI', '02', irr_low, irr_high, irr_diff)

    
    data_NOI02_1911_1920_1 = get_data_for_this_study_1(metric, '1911_1920', 'NOI', '02', irr_low, irr_high, irr_diff)
    data_NOI02_1911_1920_2 = get_data_for_this_study_2(metric, '1911_1920', 'NOI', '02', irr_low, irr_high, irr_diff)

    
    data_NOI02_1921_1930_1 = get_data_for_this_study_1(metric, '1921_1930', 'NOI', '02', irr_low, irr_high, irr_diff)
    data_NOI02_1921_1930_2 = get_data_for_this_study_2(metric, '1921_1930', 'NOI', '02', irr_low, irr_high, irr_diff)

    
    data_NOI03_1901_1910_1 = get_data_for_this_study_1(metric, '1901_1910', 'NOI', '03', irr_low, irr_high, irr_diff)
    data_NOI03_1901_1910_2 = get_data_for_this_study_2(metric, '1901_1910', 'NOI', '03', irr_low, irr_high, irr_diff)

    
    data_NOI03_1911_1920_1 = get_data_for_this_study_1(metric, '1911_1920', 'NOI', '03', irr_low, irr_high, irr_diff)
    data_NOI03_1911_1920_2 = get_data_for_this_study_2(metric, '1911_1920', 'NOI', '03', irr_low, irr_high, irr_diff)

    
    data_NOI03_1921_1930_1 = get_data_for_this_study_1(metric, '1921_1930', 'NOI', '03', irr_low, irr_high, irr_diff)
    data_NOI03_1921_1930_2 = get_data_for_this_study_2(metric, '1921_1930', 'NOI', '03', irr_low, irr_high, irr_diff)

    
    data_NOI_1901_1930 = np.concatenate((data_NOI01_1901_1910_1,
                                         data_NOI01_1901_1910_2,
                                         data_NOI01_1911_1920_1,
                                         data_NOI01_1911_1920_2,
                                         data_NOI01_1921_1930_1,
                                         data_NOI01_1921_1930_2,
                                         
                                         data_NOI02_1901_1910_1,
                                         data_NOI02_1901_1910_2,
                                         data_NOI02_1911_1920_1,
                                         data_NOI02_1911_1920_2,
                                         data_NOI02_1921_1930_1,
                                         data_NOI02_1921_1930_2,
                                         
                                         data_NOI03_1901_1910_1,
                                         data_NOI03_1901_1910_2,
                                         data_NOI03_1911_1920_1,
                                         data_NOI03_1911_1920_2,
                                         data_NOI03_1921_1930_1,
                                         data_NOI03_1921_1930_2), axis = 0)
    
    
    del data_NOI01_1901_1910_1, data_NOI01_1901_1910_2, data_NOI02_1901_1910_1, data_NOI02_1901_1910_2, data_NOI03_1901_1910_1, data_NOI03_1901_1910_2
    gc.collect()

    
    del data_NOI01_1911_1920_1, data_NOI01_1911_1920_2, data_NOI02_1911_1920_1, data_NOI02_1911_1920_2, data_NOI03_1911_1920_1, data_NOI03_1911_1920_2
    gc.collect()
    
    del data_NOI01_1921_1930_1, data_NOI01_1921_1930_2, data_NOI02_1921_1930_1, data_NOI02_1921_1930_2, data_NOI03_1921_1930_1, data_NOI03_1921_1930_2
    gc.collect()
    
    data_xr_ref = xr.DataArray(data_NOI_1901_1930, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    
    del data_NOI_1901_1930
    gc.collect()
    
    qctl=data_xr_ref.quantile(perc,dim='time')
    
    
    periods = ["1901_1910", "1911_1920", "1921_1930", "1931_1940", "1941_1950", "1951_1960", "1961_1970", "1971_1980", "1981_1984", "1985_1994", "1995_2004", "2005_2014"]
    data_time_one_year = np.array(range(2920))
    year = 0
    hour_km_2_IRR01 = np.zeros(114)
    hour_km_2_IRR02 = np.zeros(114)
    hour_km_2_IRR03 = np.zeros(114)
    hour_km_2_NOI01 = np.zeros(114)
    hour_km_2_NOI02 = np.zeros(114)
    hour_km_2_NOI03 = np.zeros(114)
    for period in periods:
        if period != "1981_1984":
            
            data_IRR01_period_1 = get_data_for_this_study_1(metric, period, 'IRR', '01', irr_low, irr_high, irr_diff)
            hour_km_2_IRR01[year:year+5] = get_hour_km2_5_years(data_IRR01_period_1, qctl)
            
            data_IRR02_period_1 = get_data_for_this_study_1(metric, period, 'IRR', '02', irr_low, irr_high, irr_diff)
            hour_km_2_IRR02[year:year+5] = get_hour_km2_5_years(data_IRR02_period_1, qctl)
            
            data_IRR03_period_1 = get_data_for_this_study_1(metric, period, 'IRR', '03', irr_low, irr_high, irr_diff)
            hour_km_2_IRR03[year:year+5] = get_hour_km2_5_years(data_IRR03_period_1, qctl)
            
            
            
            
            data_NOI01_period_1 = get_data_for_this_study_1(metric, period, 'NOI', '01', irr_low, irr_high, irr_diff)
            hour_km_2_NOI01[year:year+5] = get_hour_km2_5_years(data_NOI01_period_1, qctl)
            
            data_NOI02_period_1 = get_data_for_this_study_1(metric, period, 'NOI', '02', irr_low, irr_high, irr_diff)
            hour_km_2_NOI02[year:year+5] = get_hour_km2_5_years(data_NOI02_period_1, qctl)
            
            data_NOI03_period_1 = get_data_for_this_study_1(metric, period, 'NOI', '03', irr_low, irr_high, irr_diff)
            hour_km_2_NOI03[year:year+5] = get_hour_km2_5_years(data_NOI03_period_1, qctl)
            
            year = year + 5
            
            data_IRR01_period_2 = get_data_for_this_study_2(metric, period, 'IRR', '01', irr_low, irr_high, irr_diff)
            hour_km_2_IRR01[year:year+5] = get_hour_km2_5_years(data_IRR01_period_2, qctl)
            
            data_IRR02_period_2 = get_data_for_this_study_2(metric, period, 'IRR', '02', irr_low, irr_high, irr_diff)
            hour_km_2_IRR02[year:year+5] = get_hour_km2_5_years(data_IRR02_period_2, qctl)
            
            data_IRR03_period_2 = get_data_for_this_study_2(metric, period, 'IRR', '03', irr_low, irr_high, irr_diff)
            hour_km_2_IRR03[year:year+5] = get_hour_km2_5_years(data_IRR03_period_2, qctl)
            
            
            data_NOI01_period_2 = get_data_for_this_study_2(metric, period, 'NOI', '01', irr_low, irr_high, irr_diff)
            hour_km_2_NOI01[year:year+5] = get_hour_km2_5_years(data_NOI01_period_2, qctl)
            
            data_NOI02_period_2 = get_data_for_this_study_2(metric, period, 'NOI', '02', irr_low, irr_high, irr_diff)
            hour_km_2_NOI02[year:year+5] = get_hour_km2_5_years(data_NOI02_period_2, qctl)
            
            data_NOI03_period_2 = get_data_for_this_study_2(metric, period, 'NOI', '03', irr_low, irr_high, irr_diff)
            hour_km_2_NOI03[year:year+5] = get_hour_km2_5_years(data_NOI03_period_2, qctl)
            
            year = year + 5
            
        else:
            
            data_IRR01_period_1 = get_data_for_this_study_1(metric, period, 'IRR', '01', irr_low, irr_high, irr_diff)
            hour_km_2_IRR01[year:year+4] = get_hour_km2_4_years(data_IRR01_period_1, qctl)
            
            data_IRR02_period_1 = get_data_for_this_study_1(metric, period, 'IRR', '02', irr_low, irr_high, irr_diff)
            hour_km_2_IRR02[year:year+4] = get_hour_km2_4_years(data_IRR02_period_1, qctl)
            
            data_IRR03_period_1 = get_data_for_this_study_1(metric, period, 'IRR', '03', irr_low, irr_high, irr_diff)
            hour_km_2_IRR03[year:year+4] = get_hour_km2_4_years(data_IRR03_period_1, qctl)
            
            
            
            data_NOI01_period_1 = get_data_for_this_study_1(metric, period, 'NOI', '01', irr_low, irr_high, irr_diff)
            hour_km_2_NOI01[year:year+4] = get_hour_km2_4_years(data_NOI01_period_1, qctl)
            
            data_NOI02_period_1 = get_data_for_this_study_1(metric, period, 'NOI', '02', irr_low, irr_high, irr_diff)
            hour_km_2_NOI02[year:year+4] = get_hour_km2_4_years(data_NOI02_period_1, qctl)
            
            data_NOI03_period_1 = get_data_for_this_study_1(metric, period, 'NOI', '03', irr_low, irr_high, irr_diff)
            hour_km_2_NOI03[year:year+4] = get_hour_km2_4_years(data_NOI03_period_1, qctl)
            
            year = year + 4
                
                
    data_IRR = np.vstack((hour_km_2_IRR01, hour_km_2_IRR02, hour_km_2_IRR03))
    data_IRR_max = np.max(data_IRR, axis = 0)
    data_IRR_mean = np.mean(data_IRR, axis = 0)
    data_IRR_min = np.min(data_IRR, axis = 0)

    data_NOI = np.vstack((hour_km_2_NOI01, hour_km_2_NOI02, hour_km_2_NOI03))
    data_NOI_max = np.max(data_NOI, axis = 0)
    data_NOI_mean = np.mean(data_NOI, axis = 0)
    data_NOI_min = np.min(data_NOI, axis = 0)
    
    return data_IRR, data_NOI


In [3]:
def get_data_for_this_study_1_CESM2_gw(metric, period, exp, ens, irr_low, irr_high, irr_diff):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_1.mat', metric)
    
    data_out = np.array(data_out)
    data_out[:, irr_diff< irr_low] = np.nan
    data_out[:, irr_diff>=irr_high] = np.nan
    return data_out

def get_data_for_this_study_2_CESM2_gw(metric, period, exp, ens, irr_low, irr_high, irr_diff):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/CESM2_gw/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_2.mat', metric)
    data_out = np.array(data_out)
    data_out[:, irr_diff< irr_low] = np.nan
    data_out[:, irr_diff>=irr_high] = np.nan
    return data_out

def all_delta_irr_CESM2_gw(metric, perc, unit, variable_to_show, irr_low, irr_high):
    data_NOI01_1901_1910_1 = get_data_for_this_study_1_CESM2_gw(metric, '1901_1910', 'NOI', '01', irr_low, irr_high, irr_diff)
    data_NOI01_1901_1910_2 = get_data_for_this_study_2_CESM2_gw(metric, '1901_1910', 'NOI', '01', irr_low, irr_high, irr_diff)

    
    data_NOI01_1911_1920_1 = get_data_for_this_study_1_CESM2_gw(metric, '1911_1920', 'NOI', '01', irr_low, irr_high, irr_diff)
    data_NOI01_1911_1920_2 = get_data_for_this_study_2_CESM2_gw(metric, '1911_1920', 'NOI', '01', irr_low, irr_high, irr_diff)

    
    
    data_NOI01_1921_1930_1 = get_data_for_this_study_1_CESM2_gw(metric, '1921_1930', 'NOI', '01', irr_low, irr_high, irr_diff)
    data_NOI01_1921_1930_2 = get_data_for_this_study_2_CESM2_gw(metric, '1921_1930', 'NOI', '01', irr_low, irr_high, irr_diff)

    
    data_NOI02_1901_1910_1 = get_data_for_this_study_1_CESM2_gw(metric, '1901_1910', 'NOI', '02', irr_low, irr_high, irr_diff)
    data_NOI02_1901_1910_2 = get_data_for_this_study_2_CESM2_gw(metric, '1901_1910', 'NOI', '02', irr_low, irr_high, irr_diff)

    
    data_NOI02_1911_1920_1 = get_data_for_this_study_1_CESM2_gw(metric, '1911_1920', 'NOI', '02', irr_low, irr_high, irr_diff)
    data_NOI02_1911_1920_2 = get_data_for_this_study_2_CESM2_gw(metric, '1911_1920', 'NOI', '02', irr_low, irr_high, irr_diff)

    
    data_NOI02_1921_1930_1 = get_data_for_this_study_1_CESM2_gw(metric, '1921_1930', 'NOI', '02', irr_low, irr_high, irr_diff)
    data_NOI02_1921_1930_2 = get_data_for_this_study_2_CESM2_gw(metric, '1921_1930', 'NOI', '02', irr_low, irr_high, irr_diff)

    
    data_NOI03_1901_1910_1 = get_data_for_this_study_1_CESM2_gw(metric, '1901_1910', 'NOI', '03', irr_low, irr_high, irr_diff)
    data_NOI03_1901_1910_2 = get_data_for_this_study_2_CESM2_gw(metric, '1901_1910', 'NOI', '03', irr_low, irr_high, irr_diff)

    
    data_NOI03_1911_1920_1 = get_data_for_this_study_1_CESM2_gw(metric, '1911_1920', 'NOI', '03', irr_low, irr_high, irr_diff)
    data_NOI03_1911_1920_2 = get_data_for_this_study_2_CESM2_gw(metric, '1911_1920', 'NOI', '03', irr_low, irr_high, irr_diff)

    
    data_NOI03_1921_1930_1 = get_data_for_this_study_1_CESM2_gw(metric, '1921_1930', 'NOI', '03', irr_low, irr_high, irr_diff)
    data_NOI03_1921_1930_2 = get_data_for_this_study_2_CESM2_gw(metric, '1921_1930', 'NOI', '03', irr_low, irr_high, irr_diff)

    
    data_NOI_1901_1930 = np.concatenate((data_NOI01_1901_1910_1,
                                         data_NOI01_1901_1910_2,
                                         data_NOI01_1911_1920_1,
                                         data_NOI01_1911_1920_2,
                                         data_NOI01_1921_1930_1,
                                         data_NOI01_1921_1930_2,
                                         
                                         data_NOI02_1901_1910_1,
                                         data_NOI02_1901_1910_2,
                                         data_NOI02_1911_1920_1,
                                         data_NOI02_1911_1920_2,
                                         data_NOI02_1921_1930_1,
                                         data_NOI02_1921_1930_2,
                                         
                                         data_NOI03_1901_1910_1,
                                         data_NOI03_1901_1910_2,
                                         data_NOI03_1911_1920_1,
                                         data_NOI03_1911_1920_2,
                                         data_NOI03_1921_1930_1,
                                         data_NOI03_1921_1930_2), axis = 0)
    
    
    del data_NOI01_1901_1910_1, data_NOI01_1901_1910_2, data_NOI02_1901_1910_1, data_NOI02_1901_1910_2, data_NOI03_1901_1910_1, data_NOI03_1901_1910_2
    gc.collect()

    
    del data_NOI01_1911_1920_1, data_NOI01_1911_1920_2, data_NOI02_1911_1920_1, data_NOI02_1911_1920_2, data_NOI03_1911_1920_1, data_NOI03_1911_1920_2
    gc.collect()
    
    del data_NOI01_1921_1930_1, data_NOI01_1921_1930_2, data_NOI02_1921_1930_1, data_NOI02_1921_1930_2, data_NOI03_1921_1930_1, data_NOI03_1921_1930_2
    gc.collect()
    
    data_xr_ref = xr.DataArray(data_NOI_1901_1930, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    
    del data_NOI_1901_1930
    gc.collect()
    
    qctl=data_xr_ref.quantile(perc,dim='time')
    
    
    periods = ["1901_1910", "1911_1920", "1921_1930", "1931_1940", "1941_1950", "1951_1960", "1961_1970", "1971_1980", "1981_1984", "1985_1994", "1995_2004", "2005_2014"]
    data_time_one_year = np.array(range(2920))
    year = 0
    hour_km_2_IRR01 = np.zeros(114)
    hour_km_2_IRR02 = np.zeros(114)
    hour_km_2_IRR03 = np.zeros(114)
    hour_km_2_NOI01 = np.zeros(114)
    hour_km_2_NOI02 = np.zeros(114)
    hour_km_2_NOI03 = np.zeros(114)
    for period in periods:
        if period != "1981_1984":
            
            data_IRR01_period_1 = get_data_for_this_study_1_CESM2_gw(metric, period, 'IRR', '01', irr_low, irr_high, irr_diff)
            hour_km_2_IRR01[year:year+5] = get_hour_km2_5_years(data_IRR01_period_1, qctl)
            
            data_IRR02_period_1 = get_data_for_this_study_1_CESM2_gw(metric, period, 'IRR', '02', irr_low, irr_high, irr_diff)
            hour_km_2_IRR02[year:year+5] = get_hour_km2_5_years(data_IRR02_period_1, qctl)
            
            data_IRR03_period_1 = get_data_for_this_study_1_CESM2_gw(metric, period, 'IRR', '03', irr_low, irr_high, irr_diff)
            hour_km_2_IRR03[year:year+5] = get_hour_km2_5_years(data_IRR03_period_1, qctl)
            
            
            
            
            data_NOI01_period_1 = get_data_for_this_study_1_CESM2_gw(metric, period, 'NOI', '01', irr_low, irr_high, irr_diff)
            hour_km_2_NOI01[year:year+5] = get_hour_km2_5_years(data_NOI01_period_1, qctl)
            
            data_NOI02_period_1 = get_data_for_this_study_1_CESM2_gw(metric, period, 'NOI', '02', irr_low, irr_high, irr_diff)
            hour_km_2_NOI02[year:year+5] = get_hour_km2_5_years(data_NOI02_period_1, qctl)
            
            data_NOI03_period_1 = get_data_for_this_study_1_CESM2_gw(metric, period, 'NOI', '03', irr_low, irr_high, irr_diff)
            hour_km_2_NOI03[year:year+5] = get_hour_km2_5_years(data_NOI03_period_1, qctl)
            
            year = year + 5
            
            data_IRR01_period_2 = get_data_for_this_study_2_CESM2_gw(metric, period, 'IRR', '01', irr_low, irr_high, irr_diff)
            hour_km_2_IRR01[year:year+5] = get_hour_km2_5_years(data_IRR01_period_2, qctl)
            
            data_IRR02_period_2 = get_data_for_this_study_2_CESM2_gw(metric, period, 'IRR', '02', irr_low, irr_high, irr_diff)
            hour_km_2_IRR02[year:year+5] = get_hour_km2_5_years(data_IRR02_period_2, qctl)
            
            data_IRR03_period_2 = get_data_for_this_study_2_CESM2_gw(metric, period, 'IRR', '03', irr_low, irr_high, irr_diff)
            hour_km_2_IRR03[year:year+5] = get_hour_km2_5_years(data_IRR03_period_2, qctl)
            
            
            data_NOI01_period_2 = get_data_for_this_study_2_CESM2_gw(metric, period, 'NOI', '01', irr_low, irr_high, irr_diff)
            hour_km_2_NOI01[year:year+5] = get_hour_km2_5_years(data_NOI01_period_2, qctl)
            
            data_NOI02_period_2 = get_data_for_this_study_2_CESM2_gw(metric, period, 'NOI', '02', irr_low, irr_high, irr_diff)
            hour_km_2_NOI02[year:year+5] = get_hour_km2_5_years(data_NOI02_period_2, qctl)
            
            data_NOI03_period_2 = get_data_for_this_study_2_CESM2_gw(metric, period, 'NOI', '03', irr_low, irr_high, irr_diff)
            hour_km_2_NOI03[year:year+5] = get_hour_km2_5_years(data_NOI03_period_2, qctl)
            
            year = year + 5
            
        else:
            
            data_IRR01_period_1 = get_data_for_this_study_1_CESM2_gw(metric, period, 'IRR', '01', irr_low, irr_high, irr_diff)
            hour_km_2_IRR01[year:year+4] = get_hour_km2_4_years(data_IRR01_period_1, qctl)
            
            data_IRR02_period_1 = get_data_for_this_study_1_CESM2_gw(metric, period, 'IRR', '02', irr_low, irr_high, irr_diff)
            hour_km_2_IRR02[year:year+4] = get_hour_km2_4_years(data_IRR02_period_1, qctl)
            
            data_IRR03_period_1 = get_data_for_this_study_1_CESM2_gw(metric, period, 'IRR', '03', irr_low, irr_high, irr_diff)
            hour_km_2_IRR03[year:year+4] = get_hour_km2_4_years(data_IRR03_period_1, qctl)
            
            
            
            data_NOI01_period_1 = get_data_for_this_study_1_CESM2_gw(metric, period, 'NOI', '01', irr_low, irr_high, irr_diff)
            hour_km_2_NOI01[year:year+4] = get_hour_km2_4_years(data_NOI01_period_1, qctl)
            
            data_NOI02_period_1 = get_data_for_this_study_1_CESM2_gw(metric, period, 'NOI', '02', irr_low, irr_high, irr_diff)
            hour_km_2_NOI02[year:year+4] = get_hour_km2_4_years(data_NOI02_period_1, qctl)
            
            data_NOI03_period_1 = get_data_for_this_study_1_CESM2_gw(metric, period, 'NOI', '03', irr_low, irr_high, irr_diff)
            hour_km_2_NOI03[year:year+4] = get_hour_km2_4_years(data_NOI03_period_1, qctl)
            
            year = year + 4
                
                
    data_IRR = np.vstack((hour_km_2_IRR01, hour_km_2_IRR02, hour_km_2_IRR03))
    data_IRR_max = np.max(data_IRR, axis = 0)
    data_IRR_mean = np.mean(data_IRR, axis = 0)
    data_IRR_min = np.min(data_IRR, axis = 0)

    data_NOI = np.vstack((hour_km_2_NOI01, hour_km_2_NOI02, hour_km_2_NOI03))
    data_NOI_max = np.max(data_NOI, axis = 0)
    data_NOI_mean = np.mean(data_NOI, axis = 0)
    data_NOI_min = np.min(data_NOI, axis = 0)
    
    return data_IRR, data_NOI


In [4]:


def get_data_for_this_study_1_NorESM(metric, period, exp, ens, irr_low, irr_high, irr_diff):
    data_out_1 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_1.mat', metric)
    data_out_1 = np.array(data_out_1)
    data_out_2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_2.mat', metric)
    data_out_2 = np.array(data_out_2)
    data_out = np.concatenate((data_out_1,data_out_2),axis=0)
    del data_out_1,data_out_2
    gc.collect()
    data_out[:, irr_diff< irr_low] = np.nan
    data_out[:, irr_diff>=irr_high] = np.nan
    return data_out

def get_data_for_this_study_2_NorESM(metric, period, exp, ens, irr_low, irr_high, irr_diff):
    data_out_1 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_3.mat', metric)
    data_out_1 = np.array(data_out_1)
    data_out_2 = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_4.mat', metric)
    data_out_2 = np.array(data_out_2)
    data_out = np.concatenate((data_out_1,data_out_2),axis=0)
    del data_out_1,data_out_2
    data_out[:, irr_diff< irr_low] = np.nan
    data_out[:, irr_diff>=irr_high] = np.nan
    return data_out





def get_data_for_this_study_3_NorESM(metric, period, exp, ens, irr_low, irr_high, irr_diff):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_3.mat', metric)
    
    data_out = np.array(data_out)
    data_out[:, irr_diff< irr_low] = np.nan
    data_out[:, irr_diff>=irr_high] = np.nan
    return data_out

def get_data_for_this_study_4_NorESM(metric, period, exp, ens, irr_low, irr_high, irr_diff):
    data_out = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/3_hourly/' +\
                                             metric + '_' + period + '_3_hourly_' + exp + ens + '_4.mat', metric)
    data_out = np.array(data_out)
    data_out[:, irr_diff< irr_low] = np.nan
    data_out[:, irr_diff>=irr_high] = np.nan
    return data_out






def all_delta_irr_NorESM(metric, perc, unit, variable_to_show, irr_low, irr_high):
    data_NOI01_1901_1910_1 = get_data_for_this_study_1_NorESM(metric, '1901_1910', 'NOI', '01', irr_low, irr_high, irr_diff)
    data_NOI01_1901_1910_2 = get_data_for_this_study_2_NorESM(metric, '1901_1910', 'NOI', '01', irr_low, irr_high, irr_diff)


    
    data_NOI01_1911_1920_1 = get_data_for_this_study_1_NorESM(metric, '1911_1920', 'NOI', '01', irr_low, irr_high, irr_diff)
    data_NOI01_1911_1920_2 = get_data_for_this_study_2_NorESM(metric, '1911_1920', 'NOI', '01', irr_low, irr_high, irr_diff)


    
    
    data_NOI01_1921_1930_1 = get_data_for_this_study_1_NorESM(metric, '1921_1930', 'NOI', '01', irr_low, irr_high, irr_diff)
    data_NOI01_1921_1930_2 = get_data_for_this_study_2_NorESM(metric, '1921_1930', 'NOI', '01', irr_low, irr_high, irr_diff)


    
    data_NOI02_1901_1910_1 = get_data_for_this_study_1_NorESM(metric, '1901_1910', 'NOI', '02', irr_low, irr_high, irr_diff)
    data_NOI02_1901_1910_2 = get_data_for_this_study_2_NorESM(metric, '1901_1910', 'NOI', '02', irr_low, irr_high, irr_diff)


    
    data_NOI02_1911_1920_1 = get_data_for_this_study_1_NorESM(metric, '1911_1920', 'NOI', '02', irr_low, irr_high, irr_diff)
    data_NOI02_1911_1920_2 = get_data_for_this_study_2_NorESM(metric, '1911_1920', 'NOI', '02', irr_low, irr_high, irr_diff)


    
    data_NOI02_1921_1930_1 = get_data_for_this_study_1_NorESM(metric, '1921_1930', 'NOI', '02', irr_low, irr_high, irr_diff)
    data_NOI02_1921_1930_2 = get_data_for_this_study_2_NorESM(metric, '1921_1930', 'NOI', '02', irr_low, irr_high, irr_diff)


    
    data_NOI03_1901_1910_1 = get_data_for_this_study_1_NorESM(metric, '1901_1910', 'NOI', '03', irr_low, irr_high, irr_diff)
    data_NOI03_1901_1910_2 = get_data_for_this_study_2_NorESM(metric, '1901_1910', 'NOI', '03', irr_low, irr_high, irr_diff)


    
    data_NOI03_1911_1920_1 = get_data_for_this_study_1_NorESM(metric, '1911_1920', 'NOI', '03', irr_low, irr_high, irr_diff)
    data_NOI03_1911_1920_2 = get_data_for_this_study_2_NorESM(metric, '1911_1920', 'NOI', '03', irr_low, irr_high, irr_diff)


    
    data_NOI03_1921_1930_1 = get_data_for_this_study_1_NorESM(metric, '1921_1930', 'NOI', '03', irr_low, irr_high, irr_diff)
    data_NOI03_1921_1930_2 = get_data_for_this_study_2_NorESM(metric, '1921_1930', 'NOI', '03', irr_low, irr_high, irr_diff)


    
    data_NOI_1901_1930 = np.concatenate((data_NOI01_1901_1910_1,
                                         data_NOI01_1901_1910_2,
                                         
                                         data_NOI01_1911_1920_1,
                                         data_NOI01_1911_1920_2,
                                        
                                         data_NOI01_1921_1930_1,
                                         data_NOI01_1921_1930_2,
                                         
                                         
                                         data_NOI02_1901_1910_1,
                                         data_NOI02_1901_1910_2,
                                         
                                         data_NOI02_1911_1920_1,
                                         data_NOI02_1911_1920_2,
                                         
                                         data_NOI02_1921_1930_1,
                                         data_NOI02_1921_1930_2,
                                         
                                         
                                         data_NOI03_1901_1910_1,
                                         data_NOI03_1901_1910_2,
                                         
                                         data_NOI03_1911_1920_1,
                                         data_NOI03_1911_1920_2,
                                         
                                         data_NOI03_1921_1930_1,
                                         data_NOI03_1921_1930_2), axis = 0)
    
    
    del data_NOI01_1901_1910_1, data_NOI01_1901_1910_2, data_NOI02_1901_1910_1, data_NOI02_1901_1910_2, data_NOI03_1901_1910_1, data_NOI03_1901_1910_2
    gc.collect()
    
    
    del data_NOI01_1911_1920_1, data_NOI01_1911_1920_2, data_NOI02_1911_1920_1, data_NOI02_1911_1920_2, data_NOI03_1911_1920_1, data_NOI03_1911_1920_2
    gc.collect()
    
    
    del data_NOI01_1921_1930_1, data_NOI01_1921_1930_2, data_NOI02_1921_1930_1, data_NOI02_1921_1930_2, data_NOI03_1921_1930_1, data_NOI03_1921_1930_2
    gc.collect()
    
    data_time = np.array(range(data_NOI_1901_1930.shape[0]))
    data_xr_ref = xr.DataArray(data_NOI_1901_1930, coords={'time': data_time, 'y': data_lat, 'x': data_lon}, dims=["time", "y", "x"])
    
    del data_NOI_1901_1930
    gc.collect()
    
    qctl=data_xr_ref.quantile(perc,dim='time')
    
    
    periods = ["1901_1910", "1911_1920", "1921_1930", "1931_1940", "1941_1950", "1951_1960", "1961_1970", "1971_1980", "1981_1984", "1985_1994", "1995_2004", "2005_2014"]
    data_time_one_year = np.array(range(2920))
    year = 0
    hour_km_2_IRR01 = np.zeros(114)
    hour_km_2_IRR02 = np.zeros(114)
    hour_km_2_IRR03 = np.zeros(114)
    hour_km_2_NOI01 = np.zeros(114)
    hour_km_2_NOI02 = np.zeros(114)
    hour_km_2_NOI03 = np.zeros(114)
    for period in periods:
        if period != "1981_1984":
            
            data_IRR01_period_1 = get_data_for_this_study_1_NorESM(metric, period, 'IRR', '01', irr_low, irr_high, irr_diff)
            hour_km_2_IRR01[year:year+5] = get_hour_km2_5_years(data_IRR01_period_1, qctl)
            
            data_IRR02_period_1 = get_data_for_this_study_1_NorESM(metric, period, 'IRR', '02', irr_low, irr_high, irr_diff)
            hour_km_2_IRR02[year:year+5] = get_hour_km2_5_years(data_IRR02_period_1, qctl)
            
            data_IRR03_period_1 = get_data_for_this_study_1_NorESM(metric, period, 'IRR', '03', irr_low, irr_high, irr_diff)
            hour_km_2_IRR03[year:year+5] = get_hour_km2_5_years(data_IRR03_period_1, qctl)
            
            
            
            
            data_NOI01_period_1 = get_data_for_this_study_1_NorESM(metric, period, 'NOI', '01', irr_low, irr_high, irr_diff)
            hour_km_2_NOI01[year:year+5] = get_hour_km2_5_years(data_NOI01_period_1, qctl)
            
            data_NOI02_period_1 = get_data_for_this_study_1_NorESM(metric, period, 'NOI', '02', irr_low, irr_high, irr_diff)
            hour_km_2_NOI02[year:year+5] = get_hour_km2_5_years(data_NOI02_period_1, qctl)
            
            data_NOI03_period_1 = get_data_for_this_study_1_NorESM(metric, period, 'NOI', '03', irr_low, irr_high, irr_diff)
            hour_km_2_NOI03[year:year+5] = get_hour_km2_5_years(data_NOI03_period_1, qctl)
            
            year = year + 5
            
            data_IRR01_period_2 = get_data_for_this_study_2_NorESM(metric, period, 'IRR', '01', irr_low, irr_high, irr_diff)
            hour_km_2_IRR01[year:year+5] = get_hour_km2_5_years(data_IRR01_period_2, qctl)
            
            data_IRR02_period_2 = get_data_for_this_study_2_NorESM(metric, period, 'IRR', '02', irr_low, irr_high, irr_diff)
            hour_km_2_IRR02[year:year+5] = get_hour_km2_5_years(data_IRR02_period_2, qctl)
            
            data_IRR03_period_2 = get_data_for_this_study_2_NorESM(metric, period, 'IRR', '03', irr_low, irr_high, irr_diff)
            hour_km_2_IRR03[year:year+5] = get_hour_km2_5_years(data_IRR03_period_2, qctl)
            
            
            data_NOI01_period_2 = get_data_for_this_study_2_NorESM(metric, period, 'NOI', '01', irr_low, irr_high, irr_diff)
            hour_km_2_NOI01[year:year+5] = get_hour_km2_5_years(data_NOI01_period_2, qctl)
            
            data_NOI02_period_2 = get_data_for_this_study_2_NorESM(metric, period, 'NOI', '02', irr_low, irr_high, irr_diff)
            hour_km_2_NOI02[year:year+5] = get_hour_km2_5_years(data_NOI02_period_2, qctl)
            
            data_NOI03_period_2 = get_data_for_this_study_2_NorESM(metric, period, 'NOI', '03', irr_low, irr_high, irr_diff)
            hour_km_2_NOI03[year:year+5] = get_hour_km2_5_years(data_NOI03_period_2, qctl)
            
            year = year + 5
            
        else:
            
            data_IRR01_period_1 = get_data_for_this_study_1_NorESM(metric, period, 'IRR', '01', irr_low, irr_high, irr_diff)
            hour_km_2_IRR01[year:year+4] = get_hour_km2_4_years(data_IRR01_period_1, qctl)
            
            data_IRR02_period_1 = get_data_for_this_study_1_NorESM(metric, period, 'IRR', '02', irr_low, irr_high, irr_diff)
            hour_km_2_IRR02[year:year+4] = get_hour_km2_4_years(data_IRR02_period_1, qctl)
            
            data_IRR03_period_1 = get_data_for_this_study_1_NorESM(metric, period, 'IRR', '03', irr_low, irr_high, irr_diff)
            hour_km_2_IRR03[year:year+4] = get_hour_km2_4_years(data_IRR03_period_1, qctl)
            
            
            
            data_NOI01_period_1 = get_data_for_this_study_1_NorESM(metric, period, 'NOI', '01', irr_low, irr_high, irr_diff)
            hour_km_2_NOI01[year:year+4] = get_hour_km2_4_years(data_NOI01_period_1, qctl)
            
            data_NOI02_period_1 = get_data_for_this_study_1_NorESM(metric, period, 'NOI', '02', irr_low, irr_high, irr_diff)
            hour_km_2_NOI02[year:year+4] = get_hour_km2_4_years(data_NOI02_period_1, qctl)
            
            data_NOI03_period_1 = get_data_for_this_study_1_NorESM(metric, period, 'NOI', '03', irr_low, irr_high, irr_diff)
            hour_km_2_NOI03[year:year+4] = get_hour_km2_4_years(data_NOI03_period_1, qctl)
            
            year = year + 4
                
                
    data_IRR = np.vstack((hour_km_2_IRR01, hour_km_2_IRR02, hour_km_2_IRR03))
    data_IRR_max = np.max(data_IRR, axis = 0)
    data_IRR_mean = np.mean(data_IRR, axis = 0)
    data_IRR_min = np.min(data_IRR, axis = 0)

    data_NOI = np.vstack((hour_km_2_NOI01, hour_km_2_NOI02, hour_km_2_NOI03))
    data_NOI_max = np.max(data_NOI, axis = 0)
    data_NOI_mean = np.mean(data_NOI, axis = 0)
    data_NOI_min = np.min(data_NOI, axis = 0)
    
    return data_IRR, data_NOI

    
    
        

In [5]:
def all(var, percentile, unit, var_to_how, irr_low, irr_high, name, index):
    data_IRR_CESM2, data_NOI_CESM2=all_delta_irr(var, percentile, unit, var_to_how, irr_low, irr_high)
    data_IRR_CESM2_gw, data_NOI_CESM2_gw=all_delta_irr_CESM2_gw(var, percentile, unit, var_to_how, irr_low, irr_high)
    data_IRR_NorESM, data_NOI_NorESM=all_delta_irr_NorESM(var, percentile, unit, var_to_how, irr_low, irr_high)

    data_IRR = np.concatenate((data_IRR_CESM2,
                               data_IRR_CESM2_gw,
                               data_IRR_NorESM), axis = 0)
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/data_IRR_" + \
                     var + '_' + str(percentile) + '_' + str(irr_low) + '_' + str(irr_high) + '.mat', {'data_IRR': data_IRR})
    del data_IRR_CESM2,data_IRR_CESM2_gw,data_IRR_NorESM
    gc.collect()

    data_NOI = np.concatenate((data_NOI_CESM2,
                               data_NOI_CESM2_gw,
                               data_NOI_NorESM), axis = 0)
    scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/data_NOI_" + \
                     var + '_' + str(percentile) + '_' + str(irr_low) + '_' + str(irr_high) + '.mat', {'data_NOI': data_NOI})
    del data_NOI_CESM2,data_NOI_CESM2_gw,data_NOI_NorESM
    data_IRR_75 = np.percentile(data_IRR, 75, axis = 0)
    data_IRR_50 = np.percentile(data_IRR, 50, axis = 0)
    data_IRR_25 = np.percentile(data_IRR, 25, axis = 0)

    data_NOI_75 = np.percentile(data_NOI, 75, axis = 0)
    data_NOI_50 = np.percentile(data_NOI, 50, axis = 0)
    data_NOI_25 = np.percentile(data_NOI, 25, axis = 0)

    fig = plt.figure(figsize=(12, 6),dpi=300)
    fig.subplots_adjust(hspace=0.4, wspace=0.2, left = 0.05, right = 0.95, top = 0.95, bottom = 0.05)
    ax1 = plt.subplot(224, frameon=True)
    plt.title(name, loc='right')
    ax1.spines["top"].set_visible(False)
    ax1.spines["right"].set_visible(False)
    plot_line(ax1, index, (data_NOI_50 - data_NOI_50[0])/1000000, (data_NOI_25 - data_NOI_50[0])/1000000, (data_NOI_75 - data_NOI_50[0])/1000000, '', '', '1901irr', 'r', r'$\Delta$(km^3*hours)', 'year', '')
    plot_line(ax1, index, (data_IRR_50 - data_NOI_50[0])/1000000, (data_IRR_25 - data_NOI_50[0])/1000000, (data_IRR_75 - data_NOI_50[0])/1000000, '', '', 'tranirr', 'b', r'$\Delta$(km^3*hours)', 'year', '')


In [ ]:
all('T_s_80', 0.99, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 40, 100, '', 'a ' + r'$\Delta$irr: ' +  '>40%')
all('T_s_80', 0.995, r'$\Delta$km^3*hours', 'T_2m 99.5%', 40, 100, r'$\Delta$irr: ' +  '>40%', 'd')
all('T_s_80', 0.999, r'$\Delta$km^3*hours', 'T_2m 9', 40, 100, r'$\Delta$irr: ' +  '>40%', 'd')
all('T_s_80', 0.99, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 30, 40, '', 'a ' + r'$\Delta$irr: ' +  '>40%')
all('T_s_80', 0.995, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 30, 40, '', 'a ' + r'$\Delta$irr: ' +  '>40%')
all('T_s_80', 0.999, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 30, 40, '', 'a ' + r'$\Delta$irr: ' +  '>40%')
all('T_s_80', 0.99, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 20, 30, '', 'a ' + r'$\Delta$irr: ' +  '>40%')
all('T_s_80', 0.995, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 20, 30, '', 'a ' + r'$\Delta$irr: ' +  '>40%')
all('T_s_80', 0.99, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 0.01, 20, '', 'a ' + r'$\Delta$irr: ' +  '>40%')
all('T_s_80', 0.999, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 20, 30, '', 'a ' + r'$\Delta$irr: ' +  '>40%')
all('T_s_80', 0.995, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 0.01, 20, '', 'a ' + r'$\Delta$irr: ' +  '>40%')
all('T_s_80', 0.999, r'$\Delta$$\mathregular{10^6 km^3}$ x hours', 'T_2m 99%', 0.01, 20, '', 'a ' + r'$\Delta$irr: ' +  '>40%')